In [ ]:
# import mount to read files from the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# To Upgrade NLTK version
!pip install -U nltk 

     |████████████████████████████████| 1.5MB 24.2MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
# import libraries to read file
import pandas as pd
import numpy as np
import nltk

In [ ]:
# read the data sets
# encoding='iso-8859-1' is a parameter that can be used when working with text data in Python to specify the character encoding of the text
# it's essential to know the character encoding of the data to ensure that the text is decoded correctly
# otherwise, incorrect characters or symbols may be displayed or processed
summary = pd.read_csv('/content/drive/MyDrive/news_summary.csv', encoding='iso-8859-1')
raw = pd.read_csv('/content/drive/MyDrive/news_summary_more.csv', encoding='iso-8859-1')

In [ ]:
# shows top 5 rows of summary data  and all the attributes present in the data
summary.head()

In [ ]:
# shows top 5 rows of raw data  and all the attributes present in the data
raw.head()

In [ ]:
# creating a new DataFrame object named pre1 by selecting the first two columns of the raw DataFrame using the iloc method
# the .copy() method is used to create a copy of the selected columns instead of referencing the original DataFrame
pre1 =  raw.iloc[:,0:2].copy()
# pre1['head + text'] = pre1['headlines'].str.cat(pre1['text'], sep =" ") 


# creating a new DataFrame object named pre2 by selecting the first six columns of the summary DataFrame using the iloc method
# the str.cat() method is used to concatenate five columns (author, date, read_more, text, and ctext) in a specific order with a separator string sep
# resulting concatenated string is assigned to the text column of the pre2 DataFrame
pre2 = summary.iloc[:,0:6].copy()
pre2['text'] = pre2['author'].str.cat(pre2['date'].str.cat(pre2['read_more'].str.cat(pre2['text'].str.cat(pre2['ctext'], sep = " "), sep =" "),sep= " "), sep = " ")

In [ ]:
# creating dataframe and merging the two datasets with the same structure into a single DataFrame,
pre = pd.DataFrame()
pre['text'] = pd.concat([pre1['text'], pre2['text']], ignore_index=True)
pre['summary'] = pd.concat([pre1['headlines'],pre2['headlines']],ignore_index = True)

In [ ]:
pre_data.head(2)

,text,summary
0,"Saurav Kant, an alumnus of upGrad and IIIT-B's...",upGrad learner switches to career in ML & Al w...
1,Kunal Shah's credit card bill payment platform...,Delhi techie wins free food from Swiggy for on...


In [ ]:
# performs a series of regular expression substitutions on each row of the column to remove non-alphabetic characters and standardize the text

import re

# removes non-alphabetic characters:
def text_strip(column):
    for row in column:
        
        
        
        row=re.sub("(\\t)", ' ', str(row)).lower() # remove escape charecters
        row=re.sub("(\\r)", ' ', str(row)).lower() 
        row=re.sub("(\\n)", ' ', str(row)).lower()
        
        row=re.sub("(__+)", ' ', str(row)).lower()   # remove _ if it occors more than one time consecutively
        row=re.sub("(--+)", ' ', str(row)).lower()   # remove - if it occors more than one time consecutively
        row=re.sub("(~~+)", ' ', str(row)).lower()   # remove ~ if it occors more than one time consecutively
        row=re.sub("(\+\++)", ' ', str(row)).lower() # remove + if it occors more than one time consecutively
        row=re.sub("(\.\.+)", ' ', str(row)).lower() # remove . if it occors more than one time consecutively
        
        row=re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", ' ', str(row)).lower() # remove <>()|&©ø"',;?~*!
        
        row=re.sub("(mailto:)", ' ', str(row)).lower() # remove mailto:
        row=re.sub(r"(\\x9\d)", ' ', str(row)).lower() # remove \x9* in text
        row=re.sub("([iI][nN][cC]\d+)", 'INC_NUM', str(row)).lower() # replace INC nums to INC_NUM
        row=re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", 'CM_NUM', str(row)).lower() # replace CM# and CHG# to CM_NUM
        
        
        row=re.sub("(\.\s+)", ' ', str(row)).lower() # remove full stop at end of words(not between)
        row=re.sub("(\-\s+)", ' ', str(row)).lower() # remove - at end of words(not between)
        row=re.sub("(\:\s+)", ' ', str(row)).lower() # remove : at end of words(not between)
        
        row=re.sub("(\s+.\s+)", ' ', str(row)).lower() # remove any single charecters hanging between 2 spaces
        
        # replace any url 
        try:
            url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', str(row))
            repl_url = url.group(3)
            row = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)',repl_url, str(row))
        except:
            pass # there might be emails with no url in them
        

        
        row = re.sub("(\s+)",' ',str(row)).lower() # remove multiple spaces
        
        # should always be last
        # remove any single charecters hanging between 2 spaces
        row=re.sub("(\s+.\s+)", ' ', str(row)).lower() 

        
        
        yield row





In [ ]:
# applies the "text_strip" function to the "text" column of the "pre" DataFrame 
# and saves the cleaned text data as object named "brief_cleaning1"
brief_cleaning1 = text_strip(pre['text'])

#applies the same "text_strip" function to the "summary" column of the "pre" DataFrame
# and saves the cleaned text data as object named "brief_cleaning2"
brief_cleaning2 = text_strip(pre['summary'])

In [ ]:
from time import time
import spacy
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) 

# .pipe() method to speed-up the cleaning process
# pipeline method perform  multiple preprocessing steps into a single workflow

t = time()

# batch size refers to the number of text data points that are processed in a single iteration during training or preprocessin
# batch the data points into 5000 and run on all cores for faster preprocessing of text data
# the batch size of 5000 specifies that 5000 text data points will be processed in a single iteration of the spaCy pipeline
text = [str(doc) for doc in nlp.pipe(brief_cleaning1, batch_size=5000, n_process=-1)]

# prints the time taken for the entire cleaning process in minutes
# calculated by subtracting the current time from the start time and dividing by 60
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
# batch the data points into 5000 and run on all cores for faster preprocessing of summary data
t = time()
summary = ['_START_ '+ str(doc) + ' _END_' for doc in nlp.pipe(brief_cleaning2, batch_size=5000, n_process=-1)]
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 35.4 mins


In [ ]:
text[0]

In [ ]:
summary[0]

In [ ]:
# saving the text file of text.txt
# text.txt" in write mode and assigns the file object to the variable "f" 
# it iterates over each item in the list "text" and writes each item to a new line in the file using the "write()" method of the file object
with open('/content/drive/MyDrive/text.txt', "w",encoding = 'utf-8') as f:
  for item in text:
    f.write(item + '\n')
# saving the text file of summary.txt
with open('/content/drive/MyDrive/summary.txt', "w",encoding = 'utf-8') as f:
  for item in summary:
    f.write(item + '\n')

In [ ]:
# reads the contents of two text files, removes newline characters from each line
# and stores the processed content in two separate lists for further processing 
# encoding = 'utf-8' is used for the text file can be read and written correctly
text1 = []
summary1 = []
text = []
summary = []

with open('/content/drive/MyDrive/text.txt', "r",encoding = 'utf-8') as f:
  for line in f.readlines():
    text1.append(line)

with open('/content/drive/MyDrive/summary.txt', "r",encoding = 'utf-8') as f:
  for line in f.readlines():
    summary1.append(line)

for item in text1:
  text.append(item.replace('\n', ''))

for item in summary1:
  summary.append(item.replace('\n', ''))

In [ ]:
# adding two new columns to a Pandas DataFrame called pre
# cleaned_text and cleaned_summary is being created using the pd.Series() function to convert a Python list called text into a Pandas Series object  
pre['cleaned_text'] = pd.Series(text)
pre['cleaned_summary'] = pd.Series(summary)

In [ ]:
# creating two empty Python lists
# lists are likely to be used to store the length of each text and summary in the dataset
text_count = []
summary_count = []

In [ ]:
# calculating the word count for each text and summary in the dataset
# and storing these counts in separate lists
for sent in pre['cleaned_text']:
    text_count.append(len(sent.split()))
for sent in pre['cleaned_summary']:
    summary_count.append(len(sent.split()))

In [ ]:
# calculates the percentage of summaries 
# check how much percentage of summary have 0-15 words
cnt=0
for i in pre['cleaned_summary']:
    if(len(i.split())<=15):
        cnt=cnt+1
print(cnt/len(pre['cleaned_summary']))

0.9978234465335472


In [ ]:
# calculates the percentage of text 
# check how much percentage of text have 0-70 words
cnt=0
for i in pre['cleaned_text']:
    if(len(i.split())<=100):
        cnt=cnt+1
print(cnt/len(pre['cleaned_text']))

0.9578389933440218


In [ ]:
# model to summarize the text between 0-15 words for Summary and 0-100 words for Text
max_text_len=100
max_summary_len=15

In [ ]:
# select the Summaries and Text between max len defined above

cleaned_text =np.array(pre['cleaned_text'])
cleaned_summary=np.array(pre['cleaned_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
post_pre=pd.DataFrame({'text':short_text,'summary':short_summary})

In [ ]:
post_pre_data.head(2)

,text,summary
0,saurav kant an alumnus of upgrad and iiit-b pg...,_START_ upgrad learner switches to career in m...
1,kunal shah credit card bill payment platform c...,_START_ delhi techie wins free food from swigg...


In [ ]:
# adding special start-of-sequence (sostok) and end-of-sequence (eostok) 
# tokens to the beginning and end of each summary in the post_pre DataFrame
post_pre['summary'] = post_pre['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [ ]:
post_pre_data.head(2)

,text,summary
0,saurav kant an alumnus of upgrad and iiit-b pg...,sostok _START_ upgrad learner switches to care...
1,kunal shah credit card bill payment platform c...,sostok _START_ delhi techie wins free food fro...


In [ ]:
# Split the data to TRAIN and VALIDATION sets
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(np.array(post_pre['text']),np.array(post_pre['summary']),test_size=0.2,random_state=0,shuffle=True)

In [ ]:
from keras.preprocessing.text import Tokenizer 
from keras.utils import pad_sequences

# prepare a tokenizer for reviews on training data
#  which creates a dictionary of word index mappings based on the words in the training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_train))

In [ ]:
# RARE WORD ANALYSIS FOR X i.e 'text'
# calculates the percentage of rare words in the vocabulary and the total coverage of those rare words
# threshold,the frequency threshold for a word to be considered rare (i.e., if a word occurs less than threshold times in the corpus, it is considered rare)
# count,the count of rare words in the vocabulary (i.e., the number of words that occur less than threshold times in the corpus)
# total count,the total number of words in the vocabulary.
# frequency ,the total frequency of rare words in the corpus (i.e., the sum of the counts of all rare words)
# total_freq, the total frequency of all words in the corpu

threshold=4

count=0
total_count=0
frequency=0
total_freq=0

for key,value in x_tokenizer.word_counts.items():
    total_count=total_count+1
    total_freq=total_freq+value
    if(value<threshold):
        count=count+1
        frequency=frequency+value
    
print("% of rare words in vocabulary:",(count/total_count)*100)
print("Total Coverage of rare words:",(frequency/total_freq)*100)

% of rare words in vocabulary: 57.91270391131826
Total Coverage of rare words: 1.3404923996005096


In [ ]:
# prepare a tokenizer for reviews on training data
# tokenizer is configured to use only the top total_count - count most frequent words from the text corpus
# and any other words will be ignored
# this is done to reduce the vocabulary size and prevent overfitting
x_tokenizer = Tokenizer(num_words=total_count-count) 
x_tokenizer.fit_on_texts(list(x_train))

# convert text sequences into integer sequences 
# word in the input text is replaced with an integer, which represents its index in the vocabulary
x_train_seq    =   x_tokenizer.texts_to_sequences(x_train) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

# padding zero upto maximum length
# pad the integer sequences with zeros up to a fixed length of max_text_len
# Padding is necessary because input sequences can have varying lengths,it fixes to fixed length
x_train    =   pad_sequences(x_train_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

# size of vocabulary 
# equal to the number of unique words in the training data ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

print("Size of vocabulary in X = {}".format(x_voc))

Size of vocabulary in X = 33412


In [ ]:
# prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_train))

In [ ]:
# RARE WORD ANALYSIS FOR Y i.e 'summary'
threshold=6

count=0
total_count=0
frequency=0
total_freq=0

for key,value in y_tokenizer.word_counts.items():
    total_count=total_count+1
    total_freq=total_freq+value
    if(value<threshold):
        count=count+1
        frequency=frequency+value
    
print("% of rare words in vocabulary:",(count/total_count)*100)
print("Total Coverage of rare words:",(frequency/total_freq)*100)

% of rare words in vocabulary: 66.34503603813067
Total Coverage of rare words: 3.566630093901333


In [ ]:
# prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer(num_words=total_count-count) 
y_tokenizer.fit_on_texts(list(y_train))

# convert text sequences into integer sequences (i.e one hot encode the text in Y)
y_train_seq    =   y_tokenizer.texts_to_sequences(y_train) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

# padding zero upto maximum length
y_train   =   pad_sequences(y_train_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

# size of vocabulary
y_voc  =   y_tokenizer.num_words +1
print("Size of vocabulary in Y = {}".format(y_voc))

Size of vocabulary in Y = 11581


In [ ]:
# We will now remove "Summary" i.e Y (both train and val) which has only START and END
ind=[]
for i in range(len(y_train)):
    count=0
    for j in y_train[i]:
        if j!=0:
            count=count+1
    if(count==2):
        ind.append(i)

y_train=np.delete(y_train,ind, axis=0)
x_train=np.delete(x_train,ind, axis=0)

In [ ]:
ind=[]
for i in range(len(y_val)):
    count=0
    for j in y_val[i]:
        if j!=0:
            count=count+1
    if(count==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

In [ ]:
from keras import backend as K 
import gensim
from numpy import * 
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

print("Size of vocabulary from the w2v model = {}".format(x_voc))

K.clear_session()

# latent dimension refers to the number of dimensions in a latent space, which is a lower-dimensional space used to represent complex data such as text
# embedding dimension is used to represent each item as a vector in an embedding space that captures its properties and characteristics
latent_dim = 300
embedding_dim=200

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

# embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

# encoder lstm 1
# lstm process sequential data and capture the context and dependencies between the elements in the sequence
# encoder LSTM learns to encode the input sequence into a fixed-length vector representation
# that captures the relevant information from the entire input sequence
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

# encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

# encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

# embedding layer
# creates an embedding layer for the decoder inputs, which will map each token in the target sequence to a vector representation of size embedding_dim
# y_voc is the size of the vocabulary of the target sentence
# applies the embedding layer to the decoder inputs to obtain a 3D of shape
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

# decoder LSTM takes the encoder's output vector as input and generates the output sequence one element at a time, 
# taking into account the context and dependencies learned by the encoder
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

# dense layer
# softmax activation function ,we can obtain a probability distribution over the possible output classes/tokens, 
# which allows us to select the most likely token/class as the next element in the output sequence
# dense layer is the final layer ,it is responsible for generating the output sequence.
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()


Size of vocabulary from the w2v model = 33412
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 200)     6682400     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100, 300), ( 601200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
________________________________________________

In [ ]:
# rmsprop optimizer that adapts the learning rate based on the gradient of the loss function
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [ ]:
# monitor specifies the metric that is being monitored, which in this case is the validation loss 
# we want to minimize the validation loss, so we set mode to 'min'
# verbose argument can be useful for monitoring the progress of training (if verbose =0 no output will be printed during training if it is
#                                                                          1 means that progress updates will be printed to the console after each epoch)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

In [ ]:
# fitting the model with the data
history=model.fit([x_train,y_train[:,:-1]], y_train.reshape(y_train.shape[0],y_train.shape[1], 1)[:,1:] ,epochs=00,callbacks=[es],batch_size=128, 
                  validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Epoch 1/50
692/692 [==============================] - 801s 1s/step - loss: 5.4439 - val_loss: 4.7884
Epoch 2/50
692/692 [==============================] - 779s 1s/step - loss: 4.7358 - val_loss: 4.4829
Epoch 3/50
692/692 [==============================] - 777s 1s/step - loss: 4.4383 - val_loss: 4.2456
Epoch 4/50
692/692 [==============================] - 789s 1s/step - loss: 4.1943 - val_loss: 4.0797
Epoch 5/50
692/692 [==============================] - 781s 1s/step - loss: 4.0054 - val_loss: 3.9319
Epoch 6/50
692/692 [==============================] - 780s 1s/step - loss: 3.8555 - val_loss: 3.8262
Epoch 7/50
692/692 [==============================] - 779s 1s/step - loss: 3.7431 - val_loss: 3.7470
Epoch 8/50
692/692 [==============================] - 769s 1s/step - loss: 3.6495 - val_loss: 3.6826
Epoch 9/50
692/692 [==============================] - 767s 1s/step - loss: 3.5531 - val_loss: 3.6252
Epoch 10/50
692/692 [==============================] - 777s 1s/step - loss: 3.4722 - val_lo

In [ ]:
# saving the model weights which helps to reduce the storage space
# also we can access this without accessing the original training data 
model.save_weights('/content/drive/MyDrive/model/fitted_model')

In [ ]:
model = model.load_weights('/content/drive/MyDrive/model/fitted_model')

In [ ]:
# dictionaries that map the integer word IDs to their corresponding words in the target and source 
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index 

In [ ]:
# Encode the input sequence to get the feature vector
# the hidden state is a vector that summarizes the relevant information from the previous time steps, and it is used to inform the output at the current time step
# cell state is also a vector that is updated at each time step based on the current input and the previous cell state
# encoder outputs contains the final output of the LSTM layer, which contains information from the entire input sequence
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# creates an input layer for the decoder's hidden state vector h, with a shape of (latent_dim,)
# creates an input layer for the decoder's cell state vector c, also with a shape of (latent_dim,)
# enable the decoder to take in the encoded input text and generate a sequence of output tokens one at a time
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
# dec_emb_layer takes in decoder inputs (usually a sequence of integers representing words or tokens) and outputs their corresponding embeddings
# decoder embedding layer can be used as inputs to the decoder layers, which can then generate a sequence of output tokens based on the encoded input sequence
dec_emb2= dec_emb_layer(decoder_inputs) 

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)  #(word with the highest probability in this distribution is selected as the predicted output at each time step)

# Final decoder model
# model takes an input sequence and generates an output sequence using the initial hidden and cell states, and updates these states during the decoding process
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):
    # input sequence is encoded using the encoder model, which returns the encoded output, hidden state, and cell state
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # An empty target sequence of length 1 is created, 
    target_seq = np.zeros((1,1))
    
    # and the first word is set to the start token 'sostok'
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # decoder model predicts the next output token, given the current target sequence, encoded input, and previous hidden and cell states
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c]) 

        # Sample a token
        # predicted token with the highest probability is selected and added to the decoded sentence
        # If the selected token is not the end token, it is added to the decoded sentence and the loop continues
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # If the selected token is the end token or the maximum summary length is reached, the loop stops and return decoded sentence
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        # Internal states are used to generate the next word in the sequence using the previously generated words and the current hidden state of the model
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
# takes an input sequence as an argument and returns a string that represents a summary of the input sequence
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [ ]:
# prints out the results of these three functions for each of the first 100 training examples
for i in range(0,10):
    print("Review:",seq2text(x_train[i]))
    print("Original summary:",(seq2summary(y_train[i])).replace('start', '').replace('end', ''))
    print("Predicted summary:",(decode_sequence(x_train[i].reshape(1,max_text_len))).replace('start', '').replace('end', ''))
    print("\n")

Review: pope francis on tuesday called for respect for each ethnic group in speech delivered in myanmar avoiding reference to the rohingya minority community as the nation works to restore peace the healing of wounds must be priority he said the pope myanmar visit comes amid the country military crackdown resulting in the rohingya refugee crisis 
Original summary:  pope avoids mention of rohingyas in key myanmar speech  
Predicted summary:   pope calls for rohingya muslims in myanmar 


Review: students of government school in uttar pradesh sambhal were seen washing dishes at in school premises on being approached basic shiksha adhikari virendra pratap singh said yes have also received this complaint from elsewhere we are inquiring and action will be taken against those found guilty 
Original summary:  students seen washing dishes at govt school in up  
Predicted summary:   up school students protest up school in up 


Review: apple india profit surged by 140 in 2017 18 to crore compar

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.gleu_score import sentence_gleu
from nltk.translate.meteor_score import meteor_score

def calculate_scores(N=100):    
    bscore=0;gscore=0;mscore=0
    for i in range(N):
        ref=seq2summary(y_train[i])
        hypo=decode_sequence(x_train[i].reshape(1,max_text_len))
        bscore+=sentence_bleu([ref],hypo)
        gscore+=sentence_gleu([ref],hypo)
        mscore+=meteor_score([ref],hypo)
    print("BLEU:%.4f GLEU:%.4f METEOR:%.4f"%(bscore/N,gscore/N,mscore/N))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
calculate_scores()

BLEU:0.4279 GLEU:0.4700 METEOR:0.3422
